# IMDB Movie Review Dataset

This is a library to download and parse the [IMDB's Large Movie Review Dataset](http://ai.stanford.edu/~amaas/data/sentiment/) dataset and a demo of a transformer based model. The dataset has 25K training, and 25K test dataset, plus 50K unlabeled examples.

It's inspired on [Keras' Text classification with Transformer](https://keras.io/examples/nlp/text_classification_with_transformer/) demo.




## Environment Set Up

Let's set up `go.mod` to use the local copy of GoMLX, so it can be developed jointly the dataset code with the model. That's often how data pre-processing and model code is developed together with experimentation.

If you are not changing code, feel free to simply skip this cell. Or if you used a different directory for you projects, change it below.

Notice the directory `${HOME}/Projects/gomlx` is where the GoMLX code is copied by default in [its Docker](https://hub.docker.com/repository/docker/janpfeifer/gomlx_jupyterlab/general).

In [1]:
!*go mod edit -replace github.com/gomlx/gomlx="${HOME}/Projects/gomlx"

## Data Preparation

### Downloading data files

To download, uncompress and untar to the local directory, simply do the following. Notice if it's already downloaded in the given `--data` directory, it returns immediately.

In [2]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
    "github.com/gomlx/gomlx/ml/data"
)

var flagDataDir = flag.String("data", "~/work/imdb", "Directory to cache downloaded and generated dataset files.")

func AssertNoError(err error) {
    if err != nil {
        log.Fatalf("Failed: %+v", err)
    }
}

func AssertDownloaded() {
    *flagDataDir = data.ReplaceTildeInDir(*flagDataDir)
    if !data.FileExists(*flagDataDir) {
        AssertNoError(os.MkdirAll(*flagDataDir, 0777))
    }

    AssertNoError(imdb.Download(*flagDataDir))
}

%%
AssertDownloaded()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.


### Sampling some examples

It creates a small dataset and print out some random examples.

It also defines the `DType`, used for all internal representations of the model, and the flag `--max_len` that defines the maximum number of tokens used per observation. This will beused in the modeling later.

In [3]:
import (
    "github.com/gomlx/gomlx/examples/imdb"
)

// DType used for the models.
const DType = shapes.Float32

var (
    flagMaxLen              = flag.Int("max_len", 200, "Maximum number of tokens to take from observation.")
)

func Sample() {
    ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
    _, inputs, labels, err := ds.Yield()
    AssertNoError(err)
    labelsData := shapes.CastAsDType(labels[0].Local().FlatCopy(), shapes.Int64).([]int64)
    for ii := 0; ii < 3; ii++ {
        fmt.Printf("\nlabel=%v, input=%q\n", labelsData[ii], imdb.InputToString(inputs[0], ii))
    }
    fmt.Println()
}

%% --max_len=200
AssertDownloaded()
Sample()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.

label=0, input="<START> the most obvious flaw horrible horrible script this movie had a potentially good story but it was ruined with bad dialogue continuity problems things that were never explained gaping plotholes sub plots that went nowhere and just plain stupidity not to mention the awful cliched directing of sandra locke not even two great performances could ve saved this movie so it didn t matter that devon and rosanna arquette give horrific performances the thing is they re better actors than this movie would have you believe the best of the arquettes rosanna arquette silverado after hours desperately seeking susan has some fine moments like a great scene in the beginning when she painfully pulls her handcuffs off but gives an overall weak performance by her standards and devon gummersall dick when trumpets fade and the brill

## Training

We will create 3 different types of models for this demo: **Bag of Words** (or simply **bow**), **CNNs** and **Transformers**.

### Model Support

We first define here a few componens shared among all models:

* `Normalize` according to flag, works for sequence nodes (shaped `[batch_size, sequence_len, embedding_dim]`) and feature nodes (shaped `[batch_size, embedding_dim]`).
* `EmbedTokensGraph` that embeds the tokens, before being consumed by the models.
* `ReadoutGraph` that takes the embeddings after they are pooled on the sequence axis (so shaped `[batch_size, embedding_dim]` it adds a FNN (feed-forward neural network) with a few layers and convert to the final logits.

We also define the corresponding hyperparameters.

In [4]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/types/shapes"
)

var (
    flagNormalization    = flag.String("norm", "layer", "Type of normalization to use. Valid values are \"none\", \"batch\", \"layer\".")
    flagDropoutRate      = flag.Float64("dropout", 0.15, "Dropout rate")
    flagMaxVocab            = flag.Int("max_vocab", 20000, "Top most frequent words to consider, the rest is considered unknown.")
    flagTokenEmbeddingSize  = flag.Int("token_embed", 32, "Size of token embedding table. There are ~140K unique tokens")
    flagNumHiddenLayers     = flag.Int("hidden_layers", 2, "Number of output hidden layers, stacked with residual connection.")
    flagNumNodes            = flag.Int("num_nodes", 32, "Number of nodes in output hidden layers.")
    flagWordDropoutRate  = flag.Float64("word_dropout", 0, "Dropout rate for whole words of the input")
)

// Normalize `x` according to `--norm` flag. Works for sequence nodes (rank-3) or plain feature nodes (rank-2).
func Normalize(ctx *context.Context, x *Node) *Node {
    switch *flagNormalization {
    case "layer":
        if x.Rank() == 3 {
            // Normalize sequence.
            return layers.LayerNormalization(ctx, x, -2, -1).
                LearnedOffset(true).LearnedScale(true).ScaleNormalization(true).Done()
        } else {
            // Normalize features only.
            return layers.LayerNormalization(ctx, x, -1).Done()
        }
    case "batch":
        return layers.BatchNormalization(ctx, x, -1).Done()
    case "none":
        return x
    }
    exceptions.Panicf("invalid normalization selected %q -- valid values are batch, layer, none", *flagNormalization)
    return nil
}

// EmbedTokensGraph creates embeddings for tokens and returns them along with the mask of used tokens --
// set to false where padding was used.
func EmbedTokensGraph(ctx *context.Context, tokens *Node) (embed, mask *Node) {
    g := tokens.Graph()
    mask = NotEqual(tokens, ZerosLike(tokens)) // Mask of tokens actually used.

    // The token ids are indexed by frequency. Truncate to the vocabulary size considered, replacing
    // ids higher than that by 0.
    maxVocab := len(imdb.LoadedVocab.ListEntries)
    if maxVocab > *flagMaxVocab {
        maxVocab = *flagMaxVocab
    }

    // Limits tokens to the maxVocab.
    tokens = Where(GreaterOrEqual(tokens, Const(g, maxVocab)),
        MulScalar(OnesLike(tokens), float64(maxVocab-1)),
        tokens)

    // Embed tokens: shape=[batchSize, maxLen, embedDim]
    embed = layers.Embedding(ctx.In("tokens"), tokens, DType, maxVocab, *flagTokenEmbeddingSize)
    embed = Where(mask, embed, ZerosLike(embed))
    
    if *flagWordDropoutRate > 0 {
        dims := embed.Shape().Dimensions[:len(embed.Shape().Dimensions)-1]
        dropoutMask := Ones(g, shapes.Make(DType, dims...))
        dropoutMask = layers.Dropout(ctx, dropoutMask, ConstAsDType(g, DType, *flagWordDropoutRate))
        dropoutMask = ExpandDims(dropoutMask, -1)
        embed = Mul(embed, dropoutMask)
    }    
    return
}

// Activation function used for models.
func Activation(x *Node) *Node {
    return layers.Relu(x)
}

// ReadoutGraph takes the embeddings after they have been pooled on the sequence axis, so shaped `[batch_size, embed_dim]`
// adds a FNN on top and readout the final logit.
func ReadoutGraph(ctx *context.Context, embed *Node) *Node {
    g := embed.Graph()
    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Output layers.
    for ii := 0; ii < *flagNumHiddenLayers; ii++ {
        ctx := ctx.In(fmt.Sprintf("output_dense_%d", ii))
        residual := embed
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        //embed = Tanh(embed)
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, *flagNumNodes)
        embed = Normalize(ctx, embed)
        if ii > 0 {
            // Add residual connection.
            embed = Add(embed, residual)
        }
    }

    // Final embed layer with dimension 1.
    {
        ctx := ctx.In("readout")
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx, embed, dropoutRate)
        }
        embed = Activation(embed)
        embed = layers.DenseWithBias(ctx, embed, 1)
    }
    return embed
}

%%

### Bag Of Words Model

This is the simples model we are going to train. It's basically a no-op that connects the embedding table we defined above with the `ReadoutGraph` FNN afterwards.

We define here placeholders for our future *CNN* and *Transformer* models.

Finally we test that the shape is correct. To actually train and evaluate we still need to define the training loop, what we do in the following section.

In [5]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
)

var (
    flagModel = flag.String("model", "transformer", "Model type: bow or transformer.")
)

// ModelGraph builds the model for our demo. It returns the logits, not the predictions, which works with most losses.
func ModelGraph(ctx *context.Context, spec any, inputs []*Node) []*Node {
    _ = spec // Not used.
    tokens := inputs[0]
    embed, mask := EmbedTokensGraph(ctx, tokens)

    // Normalization function.
    if *flagModel == "bow" {
        // Bag-Of-Words model doesn't do anything, it's just the embedding table for each token.
    } else if *flagModel == "cnn" {
        embed = Conv1DGraph(ctx, embed, mask)
    } else if *flagModel == "transformer" {
        embed = TransformerGraph(ctx, tokens, embed, mask)
    } else {
        exceptions.Panicf("unknown model type %q, only types \"bow\", \"cnn\" and \"transformer\" are implemented", *flagModel)
    }

    // Sum-up per-token embeddings and do a FNN on the output. From now on, the dimensions are `[batch_dim, embed_dim]`
    // Notice we are not using mask.
    embed = ReduceMax(embed, 1)
    logits := ReadoutGraph(ctx, embed)
    return []*Node{logits}
}

func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    panic("Not implemented.")
    return nil
}

func TransformerGraph(ctx *context.Context, tokens, embed, mask *Node) *Node {
    panic("Not implemented.")
    return nil
}

%% --model=bow

// Let's test that the logits are coming out with the right shape: we want [batch_size, 10], since there are 10 classes.
AssertDownloaded()
manager := NewManager()
ds := imdb.NewDataset("Test", imdb.Test, *flagMaxLen, 3, DType, true, nil)
_, inputs, _, err := ds.Yield()
AssertNoError(err)

g := manager.NewGraph("test")
ctx := context.NewContext(manager)
logits := ModelGraph(ctx, nil, []*Node{Const(g, inputs[0])})
fmt.Printf("Logits shape for batch_size=%d: %s\n", 3, logits[0].Shape())

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Logits shape for batch_size=3: (Float32)[3 1]


### Training Loop

Training loop, with the usual bells and whistles. We also do a quick test with 100 steps to check things are working.

In [6]:
import (
    . "github.com/gomlx/gomlx/graph"
    
    "github.com/gomlx/gomlx/ml/context"
    "github.com/gomlx/gomlx/ml/train"
)

var (
    flagOptimizer        = flag.String("optimizer", "adam", "Optimizer, options: adam or sgd.")
    flagNumSteps         = flag.Int("steps", 5000, "Number of gradient descent steps to perform")
    flagBatchSize        = flag.Int("batch", 32, "Batch size for training")
    flagLearningRate     = flag.Float64("learning_rate", 0.0001, "Initial learning rate.")
    flagL2Regularization = flag.Float64("l2_reg", 0, "L2 regularization on kernels. It doesn't interact well with --batch_norm or with --optimizer=adam.")
    
    flagCheckpoint       = flag.String("checkpoint", "", "Directory save and load checkpoints from. If left empty, no checkpoints are created.")
    flagCheckpointKeep   = flag.Int("checkpoint_keep", 10, "Number of checkpoints to keep, if --checkpoint is set.")
    flagPlots            = flag.Bool("plots", true, "Plots during training: perform periodic evaluations, "+
                                     "save results if --checkpoint is set and draw plots, if in a Jupyter notebook.")
)

func trainModel() {
    // Make sure data is downloaded.
    AssertDownloaded()

    // Manager handles creation of ML computation graphs, accelerator resources, etc.
    manager := NewManager()
    fmt.Printf("Platform: %s\n", manager.Platform())

    // Datasets.
    var trainDS, trainEvalDS, testEvalDS train.Dataset
    trainDS = imdb.NewDataset("train", imdb.Train, *flagMaxLen, *flagBatchSize, DType, true, nil)
    trainEvalDS = imdb.NewDataset("train-eval", imdb.Train, *flagMaxLen, *flagBatchSize, DType, false, nil)
    testEvalDS = imdb.NewDataset("test-eval", imdb.Test, *flagMaxLen, *flagBatchSize, DType, false, nil)

    // Parallelize generation of batches, to prevent the dataset from being a bottleneck to the acceleartor (GPU/TPU).
    trainDS = data.Parallel(trainDS)
    trainEvalDS = data.Parallel(trainEvalDS)
    testEvalDS = data.Parallel(testEvalDS)

    // Metrics we are interested.
    meanAccuracyMetric := metrics.NewMeanBinaryLogitsAccuracy("Mean Accuracy", "#acc")
    movingAccuracyMetric := metrics.NewMovingAverageBinaryLogitsAccuracy("Moving Average Accuracy", "~acc", 0.01)

    // Context holds the variables and hyperparameters for the model.
    ctx := context.NewContext(manager)
    ctx.SetParam(optimizers.LearningRateKey, *flagLearningRate)
    ctx.SetParam(layers.L2RegularizationKey, *flagL2Regularization)

    // Checkpoints saving.
    var checkpoint *checkpoints.Handler
    if *flagCheckpoint != "" {
        var err error
        checkpoint, err = checkpoints.Build(ctx).DirFromBase(*flagCheckpoint, *flagDataDir).Keep(*flagCheckpointKeep).Done()
        AssertNoError(err)
    }

    // Create a train.Trainer: this object will orchestrate running the model, feeding
    // results to the optimizer, evaluating the metrics, etc. (all happens in trainer.TrainStep)
    loss := losses.BinaryCrossentropyLogits
    trainer := train.NewTrainer(
        manager, ctx, ModelGraph, loss,
        optimizers.MustOptimizerByName(*flagOptimizer),
        []metrics.Interface{movingAccuracyMetric}, // trainMetrics
        []metrics.Interface{meanAccuracyMetric})   // evalMetrics

    // Use standard training loop.
    loop := train.NewLoop(trainer)
    commandline.AttachProgressBar(loop) // Attaches a progress bar to the loop.

    // Attach a checkpoint: checkpoint every 1 minute of training.
    if checkpoint != nil {
        period := time.Minute * 1
        train.PeriodicCallback(loop, period, true, "saving checkpoint", 100,
            func(loop *train.Loop, metrics []tensor.Tensor) error {
                fmt.Printf("\n[saving checkpoint@%d] [median train step (ms): %d]\n", loop.LoopStep, loop.MedianTrainStepDuration().Milliseconds())
                return checkpoint.Save()
            })
    }

    // Attach a margaid plots: plot points at exponential steps.
    // The points generated are saved along the checkpoint directory (if one is given).
    if *flagPlots {
        _ = margaid.NewDefault(loop, checkpoint.Dir(), 100, 1.1, trainEvalDS, testEvalDS)
    }

    // Loop for given number of steps.
    _, err := loop.RunSteps(trainDS, *flagNumSteps)
    AssertNoError(err)
    fmt.Printf("\t[Step %d] median train step: %d microseconds\n",
        loop.LoopStep, loop.MedianTrainStepDuration().Microseconds())


    // Report final evaluation.
    fmt.Println()
    err = commandline.ReportEval(trainer, trainEvalDS, testEvalDS)
    AssertNoError(err)
}

%% --model=bow --steps=100 --plots=false
trainModel()


> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA
Training (100 steps):  100% [========================================] (100 steps/s) [loss=0.676] [~loss=0.688] [~acc=55.22%]        
	[Step 100] median train step: 1093 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.701
	Mean Accuracy (#acc): 49.99%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.701
	Mean Accuracy (#acc): 50.00%


### Training Bag-Of-Words ("bow") Model

A proper run of our *bow* model.

In [7]:
%% --model=bow --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (748 steps/s) [loss=0.216] [~loss=0.252] [~acc=89.69%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.244386e+00 L1.365173e+02,5.266570e+00 L2.093971e+02,5.200000e+00 L2.634977e+02,5.288788e+00 L3.073165e+02,5.244386e+00 L3.448334e+02,5.200000e+00 L3.779355e+02,5.266570e+00 L4.078967e+02,1.455362e+02 L4.355033e+02,1.531489e+02 L4.612687e+02,1.576544e+02 L4.855427e+02,1.650674e+02 L5.085726e+02,1.584312e+02 L5.306039e+02,1.673978e+02 L5.518045e+02,1.721253e+02 L5.722912e+02,1.725026e+02 L5.921466e+02,1.772522e+02 L6.114698e+02,1.780068e+02 L6.303266e+02,1.796048e+02 L6.487916e+02,1.827121e+02 L6.669114e+02,1.838884e+02 L6.847383e+02,1.846874e+02 L7.023018e+02,1.883495e+02 L7.196363e+02,1.909907e+02 L7.367580e+02,1.921448e+02 L7.536877e+02,1.940757e+02 L7.704470e+02,1.951410e+02 L7.870558e+02,1.952742e+02 L8.035318e+02,1.955405e+02 L8.198887e+02,1.941201e+02 L8.361372e+02,1.960510e+02 L8.522929e+02,1.963174e+02 L8.663200e+02,1.976046e+02 "/> <path d="M1.768000e+01,5.266570e+00 L1.365173e+02,5.244386e+00 L2.093971e+02,5.288788e+00 L2.634977e+02,5.222201e+00 L3.073165e+02,5.244386e+00 L3.448334e+02,5.288788e+00 L3.779355e+02,5.333157e+00 L4.078967e+02,1.598295e+02 L4.355033e+02,1.663325e+02 L4.612687e+02,1.725692e+02 L4.855427e+02,1.819797e+02 L5.085726e+02,1.754767e+02 L5.306039e+02,1.856196e+02 L5.518045e+02,1.911460e+02 L5.722912e+02,1.920116e+02 L5.921466e+02,1.982261e+02 L6.114698e+02,2.004899e+02 L6.303266e+02,2.031533e+02 L6.487916e+02,2.086575e+02 L6.669114e+02,2.116760e+02 L6.847383e+02,2.135847e+02 L7.023018e+02,2.179792e+02 L7.196363e+02,2.225735e+02 L7.367580e+02,2.252369e+02 L7.536877e+02,2.295648e+02 L7.704470e+02,2.323169e+02 L7.870558e+02,2.365117e+02 L8.035318e+02,2.405733e+02 L8.198887e+02,2.433254e+02 L8.361372e+02,2.490294e+02 L8.522929e+02,2.509604e+02 L8.663200e+02,2.548000e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,3.630861e+01 L1.365173e+02,2.762606e+01 L2.093971e+02,1.122032e+01 L2.634977e+02,1.944166e+01 L3.073165e+02,1.671491e+01 L3.448334e+02,1.285262e+01 L3.779355e+02,2.489019e+01 L4.078967e+02,3.880660e+01 L4.355033e+02,5.793931e+01 L4.612687e+02,7.890597e+01 L4.855427e+02,1.090309e+02 L5.085726e+02,1.169369e+02 L5.306039e+02,1.321955e+02 L5.518045e+02,1.376080e+02 L5.722912e+02,1.423853e+02 L5.921466e+02,1.496323e+02 L6.114698e+02,1.537646e+02 L6.303266e+02,1.586817e+02 L6.487916e+02,1.682157e+02 L6.669114e+02,1.778289e+02 L6.847383e+02,1.761321e+02 L7.023018e+02,1.833906e+02 L7.196363e+02,1.937787e+02 L7.367580e+02,1.940378e+02 L7.536877e+02,1.957626e+02 L7.704470e+02,2.048483e+02 L7.870558e+02,2.047085e+02 L8.035318e+02,2.108774e+02 L8.198887e+02,2.144751e+02 L8.361372e+02,2.222156e+02 L8.522929e+02,2.232216e+02 L8.663200e+02,2.253743e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps 0.500 0.600 0.700 0.800 0.900 accuracy accuracy metrics Eval on test-eval [Parallelized]: Mean Accuracy Eval on train-eval [Parallelized]: Mean Accuracy Train: Moving Average Accuracy

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.365173e+02,2.519097e+02 L2.093971e+02,2.508568e+02 L2.634977e+02,2.519883e+02 L3.073165e+02,2.507625e+02 L3.448334e+02,2.500439e+02 L3.779355e+02,2.491029e+02 L4.078967e+02,2.438217e+02 L4.355033e+02,2.282122e+02 L4.612687e+02,1.923970e+02 L4.855427e+02,1.645533e+02 L5.085726e+02,1.590336e+02 L5.306039e+02,1.488630e+02 L5.518045e+02,1.418477e+02 L5.722912e+02,1.398541e+02 L5.921466e+02,1.335911e+02 L6.114698e+02,1.291849e+02 L6.303266e+02,1.255150e+02 L6.487916e+02,1.219694e+02 L6.669114e+02,1.183647e+02 L6.847383e+02,1.150483e+02 L7.023018e+02,1.114260e+02 L7.196363e+02,1.067700e+02 L7.367580e+02,1.058904e+02 L7.536877e+02,1.027088e+02 L7.704470e+02,1.007690e+02 L7.870558e+02,9.997570e+01 L8.035318e+02,9.920353e+01 L8.198887e+02,1.011031e+02 L8.361372e+02,1.011740e+02 L8.522929e+02,1.012302e+02 L8.663200e+02,1.003557e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.547847e+02 L1.365173e+02,2.518984e+02 L2.093971e+02,2.508260e+02 L2.634977e+02,2.519509e+02 L3.073165e+02,2.506848e+02 L3.448334e+02,2.499013e+02 L3.779355e+02,2.488239e+02 L4.078967e+02,2.429846e+02 L4.355033e+02,2.256238e+02 L4.612687e+02,1.851963e+02 L4.855427e+02,1.521998e+02 L5.085726e+02,1.445649e+02 L5.306039e+02,1.326476e+02 L5.518045e+02,1.235232e+02 L5.722912e+02,1.206222e+02 L5.921466e+02,1.114729e+02 L6.114698e+02,1.042864e+02 L6.303266e+02,9.815186e+01 L6.487916e+02,9.282724e+01 L6.669114e+02,8.608324e+01 L6.847383e+02,7.936335e+01 L7.023018e+02,7.284867e+01 L7.196363e+02,6.218635e+01 L7.367580e+02,6.049176e+01 L7.536877e+02,5.341582e+01 L7.704470e+02,4.347014e+01 L7.870558e+02,3.816098e+01 L8.035318e+02,3.099725e+01 L8.198887e+02,2.796019e+01 L8.361372e+02,1.639656e+01 L8.522929e+02,1.232749e+01 L8.663200e+02,5.200000e+00 "/> <path d="M1.768000e+01,2.480921e+02 L1.365173e+02,2.496878e+02 L2.093971e+02,2.515916e+02 L2.634977e+02,2.506787e+02 L3.073165e+02,2.509270e+02 L3.448334e+02,2.508724e+02 L3.779355e+02,2.496286e+02 L4.078967e+02,2.481584e+02 L4.355033e+02,2.409230e+02 L4.612687e+02,2.272007e+02 L4.855427e+02,2.085051e+02 L5.085726e+02,1.990940e+02 L5.306039e+02,1.850663e+02 L5.518045e+02,1.766507e+02 L5.722912e+02,1.726328e+02 L5.921466e+02,1.612377e+02 L6.114698e+02,1.568946e+02 L6.303266e+02,1.504597e+02 L6.487916e+02,1.412275e+02 L6.669114e+02,1.307570e+02 L6.847383e+02,1.267392e+02 L7.023018e+02,1.200688e+02 L7.196363e+02,1.063859e+02 L7.367580e+02,1.060038e+02 L7.536877e+02,9.996819e+01 L7.704470e+02,8.831663e+01 L7.870558e+02,8.638770e+01 L8.035318e+02,7.784940e+01 L8.198887e+02,7.262576e+01 L8.361372e+02,6.086365e+01 L8.522929e+02,5.645789e+01 L8.663200e+02,5.156966e+01 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps 0.200 0.400 0.600 loss loss metrics Eval on test-eval [Parallelized]: Mean Loss Eval on train-eval [Parallelized]: Mean Loss Train: Moving Average Loss

	[Step 20000] median train step: 944 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.150
	Mean Accuracy (#acc): 94.98%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.360
	Mean Accuracy (#acc): 84.68%


### CNN Model

Below we redefine our CNN model function, with 2 convolution layers, and train on it.

Notice how well it can overfit to the training data ... but it doesn't help the test results. To improve this one needs some careful regularization.

In [8]:
// Conv1DGraph stacks two 1D convolution layers on top of the sequence embeddings (shaped `[batch_size, sequence_len, embed_dim]`).
func Conv1DGraph(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    // 1D Convolution:
    {
        ctx := ctx.In("conv1")
        embed = Activation(embed)
        embed = layers.Dropout(ctx, embed, ConstAsDType(g, DType, *flagDropoutRate))
        embed = layers.Convolution(ctx, embed).KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        embed = Normalize(ctx, embed)
    }
    {
        ctx := ctx.In("conv2")
        embed = Activation(embed)
        embed = layers.Convolution(ctx, embed).KernelSize(7).Filters(*flagTokenEmbeddingSize).Strides(3).Done()
        embed = Normalize(ctx, embed)
    }
    return embed
}

%% --model=cnn --steps=20000 --optimizer=adamw
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (20000 steps):  100% [========================================] (544 steps/s) [loss=0.080] [~loss=0.097] [~acc=96.62%]        


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,7.022230e+00 L1.365173e+02,1.294466e+01 L2.093971e+02,9.551797e+00 L2.634977e+02,7.022230e+00 L3.073165e+02,7.042297e+00 L3.448334e+02,7.644604e+00 L3.779355e+02,1.537383e+01 L4.078967e+02,1.543406e+01 L4.355033e+02,2.033260e+01 L4.612687e+02,2.103527e+01 L4.855427e+02,7.042297e+00 L5.085726e+02,3.071188e+01 L5.306039e+02,4.845904e+01 L5.518045e+02,5.472276e+01 L5.722912e+02,1.113370e+02 L5.921466e+02,1.345449e+02 L6.114698e+02,1.489393e+02 L6.303266e+02,1.558856e+02 L6.487916e+02,1.620891e+02 L6.669114e+02,1.584152e+02 L6.847383e+02,1.686339e+02 L7.023018e+02,1.686539e+02 L7.196363e+02,1.720267e+02 L7.367580e+02,1.740946e+02 L7.536877e+02,1.702600e+02 L7.704470e+02,1.684331e+02 L7.870558e+02,1.732514e+02 L8.035318e+02,1.735926e+02 L8.198887e+02,1.738135e+02 L8.361372e+02,1.696979e+02 L8.522929e+02,1.669475e+02 L8.663200e+02,1.704809e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,7.002164e+00 L1.365173e+02,1.139879e+01 L2.093971e+02,1.139879e+01 L2.634977e+02,7.042297e+00 L3.073165e+02,7.002164e+00 L3.448334e+02,7.765035e+00 L3.779355e+02,2.294249e+01 L4.078967e+02,2.406675e+01 L4.355033e+02,2.846337e+01 L4.612687e+02,3.484755e+01 L4.855427e+02,7.443815e+00 L5.085726e+02,5.086818e+01 L5.306039e+02,7.610367e+01 L5.518045e+02,7.947644e+01 L5.722912e+02,1.408086e+02 L5.921466e+02,1.669073e+02 L6.114698e+02,1.844537e+02 L6.303266e+02,1.948330e+02 L6.487916e+02,2.038471e+02 L6.669114e+02,2.028835e+02 L6.847383e+02,2.184022e+02 L7.023018e+02,2.219958e+02 L7.196363e+02,2.281391e+02 L7.367580e+02,2.345433e+02 L7.536877e+02,2.309497e+02 L7.704470e+02,2.377956e+02 L7.870558e+02,2.451836e+02 L8.035318e+02,2.481348e+02 L8.198887e+02,2.508651e+02 L8.361372e+02,2.520094e+02 L8.522929e+02,2.524712e+02 L8.663200e+02,2.548000e+02 "/> <path d="M1.768000e+01,2.379122e+01 L1.365173e+02,2.323776e+01 L2.093971e+02,1.197466e+01 L2.634977e+02,2.211389e+01 L3.073165e+02,1.783022e+01 L3.448334e+02,5.200000e+00 L3.779355e+02,1.271669e+01 L4.078967e+02,1.160281e+01 L4.355033e+02,6.243135e+00 L4.612687e+02,1.229719e+01 L4.855427e+02,1.645368e+01 L5.085726e+02,2.109700e+01 L5.306039e+02,2.965399e+01 L5.518045e+02,5.395023e+01 L5.722912e+02,9.948667e+01 L5.921466e+02,1.347708e+02 L6.114698e+02,1.574259e+02 L6.303266e+02,1.647122e+02 L6.487916e+02,1.817408e+02 L6.669114e+02,1.924121e+02 L6.847383e+02,1.990776e+02 L7.023018e+02,2.018752e+02 L7.196363e+02,2.071355e+02 L7.367580e+02,2.156630e+02 L7.536877e+02,2.155033e+02 L7.704470e+02,2.224664e+02 L7.870558e+02,2.255806e+02 L8.035318e+02,2.300711e+02 L8.198887e+02,2.342459e+02 L8.361372e+02,2.367457e+02 L8.522929e+02,2.404927e+02 L8.663200e+02,2.409504e+02 " vector-effect="non-scaling-stroke"/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps 0.500 0.600 0.700 0.800 0.900 accuracy accuracy metrics Eval on test-eval [Parallelized]: Mean Accuracy Eval on train-eval [Parallelized]: Mean Accuracy Train: Moving Average Accuracy

<path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.547189e+02 L1.365173e+02,2.517850e+02 L2.093971e+02,2.518669e+02 L2.634977e+02,2.543107e+02 L3.073165e+02,2.525985e+02 L3.448334e+02,2.520200e+02 L3.779355e+02,2.517088e+02 L4.078967e+02,2.516405e+02 L4.355033e+02,2.514492e+02 L4.612687e+02,2.514033e+02 L4.855427e+02,2.521113e+02 L5.085726e+02,2.503667e+02 L5.306039e+02,2.480008e+02 L5.518045e+02,2.401219e+02 L5.722912e+02,2.054309e+02 L5.921466e+02,1.849502e+02 L6.114698e+02,1.724509e+02 L6.303266e+02,1.627433e+02 L6.487916e+02,1.559138e+02 L6.669114e+02,1.732791e+02 L6.847383e+02,1.569444e+02 L7.023018e+02,1.571344e+02 L7.196363e+02,1.552284e+02 L7.367580e+02,1.552640e+02 L7.536877e+02,1.671947e+02 L7.704470e+02,1.814619e+02 L7.870558e+02,1.738603e+02 L8.035318e+02,1.788417e+02 L8.198887e+02,1.843396e+02 L8.361372e+02,2.103297e+02 L8.522929e+02,2.397846e+02 L8.663200e+02,2.181796e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.548000e+02 L1.365173e+02,2.519759e+02 L2.093971e+02,2.518473e+02 L2.634977e+02,2.541854e+02 L3.073165e+02,2.524283e+02 L3.448334e+02,2.517724e+02 L3.779355e+02,2.513685e+02 L4.078967e+02,2.512126e+02 L4.355033e+02,2.509261e+02 L4.612687e+02,2.508036e+02 L4.855427e+02,2.514863e+02 L5.085726e+02,2.489111e+02 L5.306039e+02,2.450145e+02 L5.518045e+02,2.306285e+02 L5.722912e+02,1.813045e+02 L5.921466e+02,1.497403e+02 L6.114698e+02,1.270506e+02 L6.303266e+02,1.118994e+02 L6.487916e+02,9.765197e+01 L6.669114e+02,9.816345e+01 L6.847383e+02,7.557851e+01 L7.023018e+02,6.851575e+01 L7.196363e+02,5.787682e+01 L7.367580e+02,4.747797e+01 L7.536877e+02,5.146502e+01 L7.704470e+02,3.982225e+01 L7.870558e+02,2.736326e+01 L8.035318e+02,2.083277e+01 L8.198887e+02,1.502209e+01 L8.361372e+02,1.206099e+01 L8.522929e+02,1.009033e+01 L8.663200e+02,5.200000e+00 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,2.528096e+02 L1.365173e+02,2.518473e+02 L2.093971e+02,2.530875e+02 L2.634977e+02,2.514748e+02 L3.073165e+02,2.522621e+02 L3.448334e+02,2.528975e+02 L3.779355e+02,2.523744e+02 L4.078967e+02,2.522403e+02 L4.355033e+02,2.521950e+02 L4.612687e+02,2.521082e+02 L4.855427e+02,2.517094e+02 L5.085726e+02,2.510834e+02 L5.306039e+02,2.497132e+02 L5.518045e+02,2.425683e+02 L5.722912e+02,2.182016e+02 L5.921466e+02,1.852698e+02 L6.114698e+02,1.615893e+02 L6.303266e+02,1.523187e+02 L6.487916e+02,1.296514e+02 L6.669114e+02,1.153844e+02 L6.847383e+02,1.088712e+02 L7.023018e+02,1.040132e+02 L7.196363e+02,9.097820e+01 L7.367580e+02,8.132325e+01 L7.536877e+02,7.770218e+01 L7.704470e+02,6.451221e+01 L7.870558e+02,5.974388e+01 L8.035318e+02,5.174267e+01 L8.198887e+02,4.696243e+01 L8.361372e+02,3.755866e+01 L8.522929e+02,3.079588e+01 L8.663200e+02,3.098911e+01 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,0 L1.768000e+01,-6 M1.287025e+02,0 L1.287025e+02,-6 M1.936465e+02,0 L1.936465e+02,-6 M2.397250e+02,0 L2.397250e+02,-6 M2.754663e+02,0 L2.754663e+02,-6 M3.046690e+02,0 L3.046690e+02,-6 M3.293596e+02,0 L3.293596e+02,-6 M3.507475e+02,0 L3.507475e+02,-6 M3.696130e+02,0 L3.696130e+02,-6 M3.864888e+02,0 L3.864888e+02,-6 M4.017547e+02,0 L4.017547e+02,-6 M4.975112e+02,0 L4.975112e+02,-6 M5.624552e+02,0 L5.624552e+02,-6 M6.085337e+02,0 L6.085337e+02,-6 M6.442750e+02,0 L6.442750e+02,-6 M6.734777e+02,0 L6.734777e+02,-6 M6.981683e+02,0 L6.981683e+02,-6 M7.195562e+02,0 L7.195562e+02,-6 M7.384217e+02,0 L7.384217e+02,-6 M7.552975e+02,0 L7.552975e+02,-6 M8.663200e+02,0 L8.663200e+02,-6 "/> 100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 4000 5000 6000 7000 8000 9000 10000 20000 Steps 0.200 0.400 0.600 loss loss metrics Eval on test-eval [Parallelized]: Mean Loss Eval on train-eval [Parallelized]: Mean Loss Train: Moving Average Loss

	[Step 20000] median train step: 1222 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.027
	Mean Accuracy (#acc): 99.38%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.602
	Mean Accuracy (#acc): 82.58%


### Transformer Model

Finally a Transformer version of the model. 

Notice it's not significantly better than our previous simple Bag-Of-Words model. Likely because there is not enough data for the transformer to make any difference. The success of transformers in large-language-models is in large part due to the training with huge amounts of unsupervised (or self-supervised) data.


In [9]:
var (
    flagMaxAttLen           = flag.Int("max_att_len", 200, "Maximum attention length: input will be split in ranges of this size.")
    flagNumAttHeads         = flag.Int("att_heads", 2, "Number of attention heads, if --model=transformer.")
    flagNumAttLayers        = flag.Int("att_layers", 1, "Number of stacked attention layers, if --model=transformer.")
    flagAttKeyQueryEmbedDim = flag.Int("att_key_dim", 8, "Dimension of the Key/Query attention embedding.")    
)

// TransformerGraph is the part of the model that takes the word/token embeddings to a tranformed
// embedding through attention ready to be pooled and read out.
func TransformerGraph(ctx *context.Context, input, embed, mask *Node) *Node {
    var newEmbed *Node
    if *flagMaxAttLen >= *flagMaxLen {
        // One transformer window covers the whole length, which makes it trivial.
        newEmbed = TransformerLayers(ctx.In("transformer"), embed, mask)
        embed = Add(embed, newEmbed)
        return embed
    }

    // Split embedding in multiple split embeddings and apply transformer in each of them.
    attLen := *flagMaxAttLen
    sequenceFrom := 0
    for {
        // x.shape = [batchSize, sequence, embedding]
        sequenceTo := sequenceFrom + attLen
        if sequenceTo > *flagMaxLen {
            sequenceTo = *flagMaxLen
            sequenceFrom = sequenceTo - attLen
        }
        // part = x[:, sequenceFrom:sequenceTo, :]
        residual := Slice(embed, AxisRange(), AxisRange(sequenceFrom, sequenceTo), AxisRange())
        partMask := Slice(mask, AxisRange(), AxisRange(sequenceFrom, sequenceTo))
        // Reuse "transformer" scope.
        part := TransformerLayers(ctx.In("transformer").Checked(false), residual, partMask)
        part = Add(residual, part)
        if newEmbed == nil {
            newEmbed = part
        } else {
            newEmbed = Add(newEmbed, part)
        }

        if sequenceTo == *flagMaxLen {
            // Reached end of parts.
            break
        }
        sequenceFrom += attLen - 20 // Attention window overlap 10 tokens among themselves.
    }
    embed = newEmbed // Notice shape changed to `[batchSize, maxAttLen, embedDim]`
    return embed
}

// TransformerLayers builds the stacked transformer layers for the model.
func TransformerLayers(ctx *context.Context, embed, mask *Node) *Node {
    g := embed.Graph()
    shape := embed.Shape()
    embedDim := shape.Dimensions[2]

    var dropoutRate *Node
    if *flagDropoutRate > 0 {
        dropoutRate = ConstAsDType(g, DType, *flagDropoutRate)
    }

    // Create positional embedding variable: it is 1 in every axis, but for the
    // sequence dimension -- there will be one embedding per position.
    // Shape: [1, maxLen, embedDim]
    posEmbedShape := shape.Copy()
    posEmbedShape.Dimensions[0] = 1
    posEmbedVar := ctx.VariableWithShape("positional", posEmbedShape)
    posEmbed := posEmbedVar.ValueGraph(g)
    embed = Add(embed, posEmbed) // Just add the embeddings, seems to work well.

    // Add the requested number of attention layers.
    for ii := 0; ii < *flagNumAttLayers; ii++ {
        // Each layer in its own scope.
        ctx := ctx.In(fmt.Sprintf("AttLayer_%d", ii))
        residual := embed
        embed = layers.MultiHeadAttention(ctx, embed, embed, embed, *flagNumAttHeads, *flagAttKeyQueryEmbedDim).
            SetKeyMask(mask).SetQueryMask(mask).
            SetOutputDim(embedDim).
            SetValueHeadDim(embedDim).Done()
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Normalize(ctx.In("normalization_1"), embed)
        attentionOutput := embed

        // Transformers recipe: 2 dense layers after attention.
        embed = layers.Dense(ctx.In("ffn_1"), embed, true, embedDim)
        embed = Tanh(embed)
        embed = layers.Dense(ctx.In("ffn_2"), embed, true, embedDim)
        if *flagDropoutRate > 0 {
            embed = layers.Dropout(ctx.In("dropout_1"), embed, dropoutRate)
        }
        embed = Add(embed, attentionOutput)
        embed = Normalize(ctx.In("normalization_2"), embed)

        // Residual connection: not part of the usual transfomer layer ...
        if ii > 0 {
            embed = Add(residual, embed)
        }
    }
    return embed
}

%% --model=transformer --steps=3000 --batch=100 --optimizer=adamw --max_len=200 --max_att_len=200 --att_heads=4 --att_layers=3 --att_key_dim=32 --max_vocab=10000
trainModel()

> Loading previously generated preprocessed binary file.
Loaded data from "aclImdb.bin": 100000 examples, 141088 unique tokens, 23727054 tokens in total.
Platform: CUDA


Training (3000 steps):  100% [========================================] (8 steps/s) [loss=0.204] [~loss=0.160] [~acc=95.14%]          


100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 Steps 0.600 0.700 0.800 0.900 accuracy accuracy metrics Eval on test-eval [Parallelized]: Mean Accuracy Eval on train-eval [Parallelized]: Mean Accuracy Train: Moving Average Accuracy

100 200 300 400 500 600 700 800 900 1000 1100 2000 3000 Steps 0.200 0.400 0.600 loss loss metrics Eval on test-eval [Parallelized]: Mean Loss Eval on train-eval [Parallelized]: Mean Loss Train: Moving Average Loss

	[Step 3000] median train step: 114828 microseconds

Results on train-eval [Parallelized]:
	Mean Loss (#loss): 0.135
	Mean Accuracy (#acc): 96.54%
Results on test-eval [Parallelized]:
	Mean Loss (#loss): 0.382
	Mean Accuracy (#acc): 85.74%
